<a href="https://colab.research.google.com/github/MUHAMMADFAZEEL01/first-try/blob/main/PROJECT_02_(RAG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain langchain-community langchain_google_genai langchain_pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.1 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata
import langchain
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/ragproject-446611-8c78fb8b1c9a.json"


In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")



In [6]:
import pinecone
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key = os.environ.get('PINECONE_API_KEY'))
spec=ServerlessSpec(cloud="aws", region="us-east-1")

In [ ]:

if 'rag' not in pc.list_indexes():
  pc.create_index(
      name =  'rag' ,
      dimension =768,
      metric ='cosine',
      spec = spec
  )


In [8]:
index = pc.Index('rag')

In [9]:
pc.describe_index('rag')

{'deletion_protection': 'disabled',
 'dimension': 768,
 'host': 'rag-asi64lk.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'rag',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

In [10]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import pdf
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [11]:
!pip install --upgrade --quiet pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.5 MB/s eta 0:00:00


In [12]:
from langchain_community.document_loaders import PyPDFLoader

filepdf= (
    '/content/Strategic Mindset  A 7-Day Plan to Identify What Matters and Create a Strategy that Works (Productivity Series Book 4) (Thibaut  Meurisse) (Z-Library).pdf'
)
loader = PyPDFLoader(filepdf)


In [13]:
documents =loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [16]:
for doc in docs:
    # Generate vector for the document
    vector = embeddings.embed_query(doc.page_content)
    metadata = {"source": doc.metadata.get("source", "unknown")}
    index.upsert([(doc.metadata['source'], vector, metadata)])  # (id, vector, metadata)



In [17]:
retriever = Pinecone(index=index, embedding=embeddings, text_key="text")

In [18]:
from langchain_pinecone import PineconeVectorStore

In [19]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [20]:
vector_store = PineconeVectorStore(
    index=index,
    embedding=embeddings,
    text_key="text",
)

In [21]:
results = vector_store.similarity_search(
    "purpose of this book?",
    k=2,
)



In [25]:
from IPython.display import Markdown, display
for res in results:
     p = res.page_content
     display(Markdown(p))




Then, using the content from this book, develop the mindset of the strategic
producer. Whenever you face a new task, ask yourself whether it’s really
the best use of your time. If you do this consistently, over time, you’ll
become one of the best strategists you know. More importantly, you’ll reach
most of your goals, even the ones that seem difficult or perhaps impossible
right now.
It’s now up to you to perform the work and achieve the results you aspire
to. I can’t do the work for you, but I can wish you all the best in your future
endeavors.
Thibaut Meurisse.

Then, using the content from this book, develop the mindset of the strategic
producer. Whenever you face a new task, ask yourself whether it’s really
the best use of your time. If you do this consistently, over time, you’ll
become one of the best strategists you know. More importantly, you’ll reach
most of your goals, even the ones that seem difficult or perhaps impossible
right now.
It’s now up to you to perform the work and achieve the results you aspire
to. I can’t do the work for you, but I can wish you all the best in your future
endeavors.
Thibaut Meurisse.

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(api_key=os.environ.get('GEMINI_API_KEY'),temperature =0, model="gemini-1.5-flash", max_retries=2)

In [31]:
def main(query,vector_store):

  results = vector_store.similarity_search(query,k=3)
  context = '\n'.join([res.page_content for res in results])

  response = model.invoke( f" You're AI asistence in the below context is given please give me answer of the query in which is context: {context} query: {query} ")
  return response

question =str( input(" Write a question which is realted to the Strategic Mindset  A 7-Day Plan"))
result = main(question,vector_store)
display(Markdown(result.content))

 Write a question which is realted to the Strategic Mindset  A 7-Day Plandescribe the purpose of this book and if i read this book what can i achieve in my life with the help of this book


The purpose of the book is to help you develop a highly effective strategy to achieve your goals.  It's designed to be used in conjunction with a workbook (available for download at https://whatispersonaldevelopment.org/strategic-mindset or at the end of the book).

By reading this book and completing the workbook, you can achieve a more strategic mindset, enabling you to better plan and execute your goals in life.  The exact achievements will depend on your individual goals, but the book aims to equip you with the tools and framework to make significant progress towards whatever you want to accomplish.
